In [1]:
# import modules
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from pandas.io import sql
import datetime
import pandas as pd
import pandasql as ps

In [3]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
five2 = pd.read_csv('./final/five/temp/mid_result_of_five2.txt', sep='\t')

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-07 14:05:35.612547
종료: 2024-03-07 14:06:13.111021


In [4]:
five2

,cluster_id,pubyear,uid,subject_traditional,sub_n,w,unique_uid_count,weight_cate,norm_weight_cate
0,1,2006,WOS:000241489600007,Geochemistry & Geophysics,1,1.00,52314,13212.75,0.252566
1,1,2006,WOS:000235430800004,Geochemistry & Geophysics,1,1.00,52314,13212.75,0.252566
2,1,2006,WOS:000236201900004,Geochemistry & Geophysics,2,0.50,52314,13212.75,0.252566
3,1,2006,WOS:000236600600022,Geochemistry & Geophysics,1,1.00,52314,13212.75,0.252566
4,1,2006,WOS:000240601800015,Geochemistry & Geophysics,1,1.00,52314,13212.75,0.252566
...,...,...,...,...,...,...,...,...,...
41190333,4215,2020,WOS:000580351000001,"Physics, Applied",4,0.25,71,0.50,0.007042
41190334,4215,2019,WOS:000511959000012,Metallurgy & Metallurgical Engineering,1,1.00,71,1.00,0.014085
41190335,4215,2020,WOS:000580351000001,"Chemistry, Analytical",4,0.25,71,0.25,0.003521
41190336,4215,2020,WOS:000580351000001,Nanoscience & Nanotechnology,4,0.25,71,0.25,0.003521


In [7]:
five2.drop(['unique_uid_count', 'weight_cate', 'norm_weight_cate'], axis = 1, inplace = True)

In [8]:
five2

,cluster_id,pubyear,uid,subject_traditional,sub_n,w
0,1,2006,WOS:000241489600007,Geochemistry & Geophysics,1,1.00
1,1,2006,WOS:000235430800004,Geochemistry & Geophysics,1,1.00
2,1,2006,WOS:000236201900004,Geochemistry & Geophysics,2,0.50
3,1,2006,WOS:000236600600022,Geochemistry & Geophysics,1,1.00
4,1,2006,WOS:000240601800015,Geochemistry & Geophysics,1,1.00
...,...,...,...,...,...,...
41190333,4215,2020,WOS:000580351000001,"Physics, Applied",4,0.25
41190334,4215,2019,WOS:000511959000012,Metallurgy & Metallurgical Engineering,1,1.00
41190335,4215,2020,WOS:000580351000001,"Chemistry, Analytical",4,0.25
41190336,4215,2020,WOS:000580351000001,Nanoscience & Nanotechnology,4,0.25


### 컬럼
sub_n : uid 당 cate 개수  
w = 1 / sub_n  
weight_cate = 한 클러스터 안에서 카테고리 별 각각 w를 합산한 값  
norm_weight_cate = weight_cate를 속한 클러스터의 unique uid 개수로 나눈 값  

In [10]:
# Assuming sorted_five is your dataframe and it already exists with a column named 'sub_n'
# Example to add a new column 'w' where w = 1 / sub_n

#sorted_five['w'] = 1 / sorted_five['sub_n']

In [10]:
# 클러스터당 uid unique 개수 카운트
# df_example 데이터프레임을 사용하여 cluster_id + pubyear 별로 uid의 유니크한 개수를 카운트합니다.
uid_counts_per_cluster = five2.groupby(['cluster_id', 'pubyear'])['uid'].nunique().reset_index(name='unique_uid_count')

# 원래 df_example 데이터프레임에 uid_counts_per_cluster 데이터프레임을 merge하여,
# 각 row에 대한 cluster_id별 unique uid 개수를 추가합니다.
sorted_five = pd.merge(five2, uid_counts_per_cluster, on=['cluster_id', 'pubyear'])




In [11]:

# cluster_id와 subject_traditional로 groupby 하고, w 컬럼의 합계를 계산 후,
# 이를 원본 df_example에 merge하여 각 row에 대한 weight_cate 값을 추가합니다.
grouped_five = sorted_five.groupby(['cluster_id', 'pubyear', 'subject_traditional'])['w'].sum().reset_index(name='weight_cate')

# 원본 데이터프레임에 weight_cate 값을 merge합니다.
five_result1 = pd.merge(sorted_five, grouped_five, on=['cluster_id', 'pubyear', 'subject_traditional'])




In [12]:
five_result1

,cluster_id,pubyear,uid,subject_traditional,sub_n,w,unique_uid_count,weight_cate
0,1,2006,WOS:000241489600007,Geochemistry & Geophysics,1,1.000000,2590,689.000000
1,1,2006,WOS:000235430800004,Geochemistry & Geophysics,1,1.000000,2590,689.000000
2,1,2006,WOS:000236201900004,Geochemistry & Geophysics,2,0.500000,2590,689.000000
3,1,2006,WOS:000236600600022,Geochemistry & Geophysics,1,1.000000,2590,689.000000
4,1,2006,WOS:000240601800015,Geochemistry & Geophysics,1,1.000000,2590,689.000000
...,...,...,...,...,...,...,...,...
41190333,4215,2021,WOS:000592049600001,Operations Research & Management Science,3,0.333333,6,0.333333
41190334,4215,2021,WOS:000664877800004,"Mathematics, Interdisciplinary Applications",2,0.500000,6,0.500000
41190335,4215,2021,WOS:000622690800001,"Engineering, Mechanical",2,0.500000,6,1.000000
41190336,4215,2021,WOS:000689390100001,"Engineering, Mechanical",2,0.500000,6,1.000000


In [13]:
five_result1['norm_weight_cate'] = five_result1['weight_cate'] / sorted_five['unique_uid_count']

In [14]:
five_result1

,cluster_id,pubyear,uid,subject_traditional,sub_n,w,unique_uid_count,weight_cate,norm_weight_cate
0,1,2006,WOS:000241489600007,Geochemistry & Geophysics,1,1.000000,2590,689.000000,0.266023
1,1,2006,WOS:000235430800004,Geochemistry & Geophysics,1,1.000000,2590,689.000000,0.266023
2,1,2006,WOS:000236201900004,Geochemistry & Geophysics,2,0.500000,2590,689.000000,0.266023
3,1,2006,WOS:000236600600022,Geochemistry & Geophysics,1,1.000000,2590,689.000000,0.266023
4,1,2006,WOS:000240601800015,Geochemistry & Geophysics,1,1.000000,2590,689.000000,0.266023
...,...,...,...,...,...,...,...,...,...
41190333,4215,2021,WOS:000592049600001,Operations Research & Management Science,3,0.333333,6,0.333333,0.055556
41190334,4215,2021,WOS:000664877800004,"Mathematics, Interdisciplinary Applications",2,0.500000,6,0.500000,0.083333
41190335,4215,2021,WOS:000622690800001,"Engineering, Mechanical",2,0.500000,6,1.000000,0.166667
41190336,4215,2021,WOS:000689390100001,"Engineering, Mechanical",2,0.500000,6,1.000000,0.166667


In [15]:
now = datetime.datetime.now() 
print('시작:', now)

five_result1.to_csv('./final/five/temp/five2_result1.txt', sep='\t', index=False)

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-07 14:18:22.701626
종료: 2024-03-07 14:21:37.815715


1. final_result1에서 cluster_id와 subject_tra를 groupby하기  
2.필요 컬럼만 추출하여 저장 = 5-1번

In [16]:

# 'five_result1' 데이터프레임에서 'cluster_id'와 'subject_traditional'로 groupby한 후, 각 그룹의 첫 번째 행을 선택
grouped_df = five_result1.groupby(['cluster_id', 'pubyear',  'subject_traditional']).first().reset_index()

In [17]:
grouped_df

,cluster_id,pubyear,subject_traditional,uid,sub_n,w,unique_uid_count,weight_cate,norm_weight_cate
0,1,2006,Anthropology,WOS:000241072600005,2,0.500000,2590,0.500000,0.000193
1,1,2006,Archaeology,WOS:000241072600005,2,0.500000,2590,0.500000,0.000193
2,1,2006,Astronomy & Astrophysics,WOS:000238380600016,2,0.500000,2590,0.500000,0.000193
3,1,2006,Biochemical Research Methods,WOS:000235755000028,3,0.333333,2590,0.333333,0.000129
4,1,2006,"Chemistry, Analytical",WOS:000235990200006,2,0.500000,2590,8.166667,0.003153
...,...,...,...,...,...,...,...,...,...
2627423,4215,2021,"Engineering, Mechanical",WOS:000622690800001,2,0.500000,6,1.000000,0.166667
2627424,4215,2021,"Mathematics, Applied",WOS:000661166500001,3,0.333333,6,1.333333,0.222222
2627425,4215,2021,"Mathematics, Interdisciplinary Applications",WOS:000664877800004,2,0.500000,6,0.500000,0.083333
2627426,4215,2021,Multidisciplinary Sciences,WOS:000664877800004,2,0.500000,6,0.500000,0.083333


In [22]:
# Selecting specific columns from the grouped DataFrame
selected_columns_df = grouped_df[['cluster_id', 'pubyear', 'subject_traditional', 'weight_cate', 'norm_weight_cate']]



In [23]:
selected_columns_df

,cluster_id,pubyear,subject_traditional,weight_cate,norm_weight_cate
0,1,2006,Anthropology,0.500000,0.000193
1,1,2006,Archaeology,0.500000,0.000193
2,1,2006,Astronomy & Astrophysics,0.500000,0.000193
3,1,2006,Biochemical Research Methods,0.333333,0.000129
4,1,2006,"Chemistry, Analytical",8.166667,0.003153
...,...,...,...,...,...
2627423,4215,2021,"Engineering, Mechanical",1.000000,0.166667
2627424,4215,2021,"Mathematics, Applied",1.333333,0.222222
2627425,4215,2021,"Mathematics, Interdisciplinary Applications",0.500000,0.083333
2627426,4215,2021,Multidisciplinary Sciences,0.500000,0.083333


In [24]:
now = datetime.datetime.now() 
print('시작:', now)

selected_columns_df.to_csv('./final/five/temp/five2_result2.txt', sep='\t', index=False)

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-07 14:24:01.907851
종료: 2024-03-07 14:24:10.821855


In [25]:
unique_counts = selected_columns_df.groupby(['cluster_id', 'pubyear'])['subject_traditional'].nunique()

# 결과 출력
print(unique_counts)

cluster_id  pubyear
1           2006       44
            2007       44
            2008       40
            2009       35
            2010       39
                       ..
4215        2017        7
            2018        6
            2019        6
            2020        8
            2021        8
Name: subject_traditional, Length: 67382, dtype: int64


In [26]:
import pickle
# Pickle 파일에서 리스트 불러오기
with open('./mid/unique_subjects.pkl', 'rb') as file:
    loaded_unique_subjects = pickle.load(file)

# 이제 loaded_unique_subjects 리스트를 사용하여 DataFrame의 컬럼을 생성할 수 있음


In [27]:
print(len(loaded_unique_subjects))

254


## 뼈대 다시 생각해서 만들기

1. 뼈대 만들기  
2. left join하기 - 결측값은 0으로 채우기  

In [29]:
import pandas as pd
import numpy as np

# cluster_id 생성: 1부터 4215까지
#cluster_ids = np.arange(1, 4216)

# 기존뼈대 가지고 오기
df_basic = selected_columns_df[['cluster_id', 'pubyear']]

In [30]:
df_basic

,cluster_id,pubyear
0,1,2006
1,1,2006
2,1,2006
3,1,2006
4,1,2006
...,...,...
2627423,4215,2021
2627424,4215,2021
2627425,4215,2021
2627426,4215,2021


In [31]:

# 'five_result1' 데이터프레임에서 'cluster_id'와 'subject_traditional'로 groupby한 후, 각 그룹의 첫 번째 행을 선택
df_basic = df_basic.groupby(['cluster_id', 'pubyear']).first().reset_index()

In [32]:
df_basic

,cluster_id,pubyear
0,1,2006
1,1,2007
2,1,2008
3,1,2009
4,1,2010
...,...,...
67377,4215,2017
67378,4215,2018
67379,4215,2019
67380,4215,2020


레프트 조인

In [33]:
subject_traditional = loaded_unique_subjects

In [34]:
print(len(subject_traditional))

254


In [35]:
# df_basic를 subject_traditional 리스트의 길이만큼 반복
df_repeated = pd.DataFrame(np.repeat(df_basic.values, len(subject_traditional), axis=0))
df_repeated.columns = df_basic.columns

# 새로운 'subject_traditional' 컬럼을 생성하여 반복된 DataFrame에 추가
df_repeated['subject_traditional'] = pd.Series(subject_traditional * len(df_basic))


In [37]:
df_repeated = df_repeated.sort_values(by=['cluster_id','pubyear', 'subject_traditional'])

In [38]:
df_repeated

,cluster_id,pubyear,subject_traditional
92,1,2006,Acoustics
240,1,2006,Agricultural Economics & Policy
232,1,2006,Agricultural Engineering
159,1,2006,"Agriculture, Dairy & Animal Science"
161,1,2006,"Agriculture, Multidisciplinary"
...,...,...,...
17114907,4215,2021,Veterinary Sciences
17114884,4215,2021,Virology
17114788,4215,2021,Water Resources
17114979,4215,2021,Women's Studies


In [40]:
unique_counts = df_repeated.groupby(['cluster_id','pubyear'])['subject_traditional'].nunique()

# 결과 출력
print(unique_counts)

cluster_id  pubyear
1           2006       254
            2007       254
            2008       254
            2009       254
            2010       254
                      ... 
4215        2017       254
            2018       254
            2019       254
            2020       254
            2021       254
Name: subject_traditional, Length: 67382, dtype: int64


In [41]:

now = datetime.datetime.now() 
print('시작:', now)

result= pd.merge(df_repeated, selected_columns_df, on=['cluster_id','pubyear', 'subject_traditional'], how='left')

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-07 14:51:47.588657
종료: 2024-03-07 14:51:51.075445


In [42]:
result

,cluster_id,pubyear,subject_traditional,weight_cate,norm_weight_cate
0,1,2006,Acoustics,NaN,NaN
1,1,2006,Agricultural Economics & Policy,NaN,NaN
2,1,2006,Agricultural Engineering,NaN,NaN
3,1,2006,"Agriculture, Dairy & Animal Science",NaN,NaN
4,1,2006,"Agriculture, Multidisciplinary",NaN,NaN
...,...,...,...,...,...
17115023,4215,2021,Veterinary Sciences,NaN,NaN
17115024,4215,2021,Virology,NaN,NaN
17115025,4215,2021,Water Resources,NaN,NaN
17115026,4215,2021,Women's Studies,NaN,NaN


In [43]:
result.fillna(0, inplace=True)

In [44]:
result

,cluster_id,pubyear,subject_traditional,weight_cate,norm_weight_cate
0,1,2006,Acoustics,0.0,0.0
1,1,2006,Agricultural Economics & Policy,0.0,0.0
2,1,2006,Agricultural Engineering,0.0,0.0
3,1,2006,"Agriculture, Dairy & Animal Science",0.0,0.0
4,1,2006,"Agriculture, Multidisciplinary",0.0,0.0
...,...,...,...,...,...
17115023,4215,2021,Veterinary Sciences,0.0,0.0
17115024,4215,2021,Virology,0.0,0.0
17115025,4215,2021,Water Resources,0.0,0.0
17115026,4215,2021,Women's Studies,0.0,0.0


In [45]:
# 'cluster_id'를 우선적으로, 그리고 'subject_traditional'을 알파벳 순으로 정렬
result = result.sort_values(by=['cluster_id','pubyear','subject_traditional'])


In [46]:
result.to_csv('./final/five/temp/five2_result3.txt', sep='\t', index=False)

In [47]:
result

,cluster_id,pubyear,subject_traditional,weight_cate,norm_weight_cate
0,1,2006,Acoustics,0.0,0.0
1,1,2006,Agricultural Economics & Policy,0.0,0.0
2,1,2006,Agricultural Engineering,0.0,0.0
3,1,2006,"Agriculture, Dairy & Animal Science",0.0,0.0
4,1,2006,"Agriculture, Multidisciplinary",0.0,0.0
...,...,...,...,...,...
17115023,4215,2021,Veterinary Sciences,0.0,0.0
17115024,4215,2021,Virology,0.0,0.0
17115025,4215,2021,Water Resources,0.0,0.0
17115026,4215,2021,Women's Studies,0.0,0.0


## wide형 데이터 프레임 만들기

이 접근 방법은 cluster_id와 pubyear의 조합으로 고유한 식별자를 만들어, 각 조합별로 subject_traditional에 대한 norm_weight_cate 값을 정리합니다. 이후 필요에 따라 고유 식별자를 다시 분리하여 원래의 cluster_id와 pubyear 정보를 복원할 수 있습니다.

단계별 접근  
고유 식별자 생성: cluster_id와 pubyear를 결합하여 고유 식별자를 만듭니다.  
Pivot 작업: 생성한 고유 식별자를 인덱스로 사용하고, subject_traditional을 컬럼으로 사용하여 pivot_table을 수행합니다.  
고유 식별자 분리: 필요한 경우, 결과 데이터프레임에서 고유 식별자를 분리하여 원래의 cluster_id와 pubyear를 복원합니다.  

In [93]:
df = result

In [94]:
import pandas as pd

# 고유 식별자 생성
df['unique_id'] = df['cluster_id'].astype(str) + '_' + df['pubyear'].astype(str)

In [88]:
df

,cluster_id,pubyear,subject_traditional,weight_cate,norm_weight_cate,unique_id
0,1,2006,Acoustics,0.0,0.0,1_2006
1,1,2006,Agricultural Economics & Policy,0.0,0.0,1_2006
2,1,2006,Agricultural Engineering,0.0,0.0,1_2006
3,1,2006,"Agriculture, Dairy & Animal Science",0.0,0.0,1_2006
4,1,2006,"Agriculture, Multidisciplinary",0.0,0.0,1_2006
...,...,...,...,...,...,...
17115023,4215,2021,Veterinary Sciences,0.0,0.0,4215_2021
17115024,4215,2021,Virology,0.0,0.0,4215_2021
17115025,4215,2021,Water Resources,0.0,0.0,4215_2021
17115026,4215,2021,Women's Studies,0.0,0.0,4215_2021


In [95]:
# Pivot 작업
pivot_df = df.pivot_table(index='unique_id', 
                          columns='subject_traditional', 
                          values='norm_weight_cate', 
                          fill_value=0)
pivot_df.reset_index(inplace=True)

In [97]:
# unique_id 컬럼을 '_'를 기준으로 분리하여 cluster_id와 pubyear로 나눕니다.
# 그 후, 각각을 정수형으로 변환합니다.


# unique_id 컬럼을 분리하고 정수형으로 변환
pivot_df[['cluster_id', 'pubyear']] = pivot_df['unique_id'].str.split('_', expand=True)
pivot_df['cluster_id'] = pivot_df['cluster_id'].astype(int)
pivot_df['pubyear'] = pivot_df['pubyear'].astype(int)

# 변환된 임시 컬럼을 사용하여 데이터프레임을 정렬합니다.
pivot_df_sorted = pivot_df.sort_values(by=['cluster_id', 'pubyear'])

# 정렬된 데이터프레임을 확인합니다.
print(pivot_df_sorted)


subject_traditional  unique_id  Acoustics  Agricultural Economics & Policy  \
17786                   1_2006        0.0                              0.0   
17787                   1_2007        0.0                              0.0   
17788                   1_2008        0.0                              0.0   
17789                   1_2009        0.0                              0.0   
17790                   1_2010        0.0                              0.0   
...                        ...        ...                              ...   
57070                4215_2017        0.0                              0.0   
57071                4215_2018        0.0                              0.0   
57072                4215_2019        0.0                              0.0   
57073                4215_2020        0.0                              0.0   
57074                4215_2021        0.0                              0.0   

subject_traditional  Agricultural Engineering  \
17786         

In [98]:
pivot_df_sorted

subject_traditional,unique_id,Acoustics,Agricultural Economics & Policy,Agricultural Engineering,"Agriculture, Dairy & Animal Science","Agriculture, Multidisciplinary",Agronomy,Allergy,Anatomy & Morphology,Andrology,...,Tropical Medicine,Urban Studies,Urology & Nephrology,Veterinary Sciences,Virology,Water Resources,Women's Studies,Zoology,cluster_id,pubyear
17786,1_2006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000129,0.0,0.0,1,2006
17787,1_2007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000308,0.0,0.0,1,2007
17788,1_2008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000368,0.0,0.0,1,2008
17789,1_2009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1,2009
17790,1_2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000125,0.0,0.0,1,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57070,4215_2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,4215,2017
57071,4215_2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,4215,2018
57072,4215_2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,4215,2019
57073,4215_2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,4215,2020


In [100]:


# 원하는 컬럼 순서 생성: 'cluster_id' 다음에 'pubyear'가 오도록
new_columns_order = ['cluster_id', 'pubyear'] + [col for col in pivot_df.columns if col not in [ 'cluster_id', 'pubyear']]

# DataFrame의 컬럼을 새 순서로 재배열
pivot_df_1 = pivot_df_sorted[new_columns_order]

# 소트

pivot_df_1 = pivot_df_1.sort_values(by=['cluster_id','pubyear'])

In [101]:
pivot_df_1

subject_traditional,cluster_id,pubyear,unique_id,Acoustics,Agricultural Economics & Policy,Agricultural Engineering,"Agriculture, Dairy & Animal Science","Agriculture, Multidisciplinary",Agronomy,Allergy,...,Transportation,Transportation Science & Technology,Tropical Medicine,Urban Studies,Urology & Nephrology,Veterinary Sciences,Virology,Water Resources,Women's Studies,Zoology
17786,1,2006,1_2006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000129,0.0,0.0
17787,1,2007,1_2007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000308,0.0,0.0
17788,1,2008,1_2008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000368,0.0,0.0
17789,1,2009,1_2009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
17790,1,2010,1_2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000125,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57070,4215,2017,4215_2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
57071,4215,2018,4215_2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
57072,4215,2019,4215_2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
57073,4215,2020,4215_2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [103]:
pivot_df_2= pivot_df_1.drop(['unique_id'], axis = 1, inplace = False)

In [104]:
pivot_df_2

subject_traditional,cluster_id,pubyear,Acoustics,Agricultural Economics & Policy,Agricultural Engineering,"Agriculture, Dairy & Animal Science","Agriculture, Multidisciplinary",Agronomy,Allergy,Anatomy & Morphology,...,Transportation,Transportation Science & Technology,Tropical Medicine,Urban Studies,Urology & Nephrology,Veterinary Sciences,Virology,Water Resources,Women's Studies,Zoology
17786,1,2006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000129,0.0,0.0
17787,1,2007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000308,0.0,0.0
17788,1,2008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000368,0.0,0.0
17789,1,2009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
17790,1,2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000125,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57070,4215,2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
57071,4215,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
57072,4215,2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
57073,4215,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [106]:
pivot_df_2.to_csv('./final/five/temp/five2_result4.txt', sep='\t', index=False)

In [108]:
num_columns = len(pivot_df_2.columns)

num_columns

256